In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from collections import defaultdict

In [2]:
# Load the dataset
file_path = 'D:/aip/dataset_withseparatedmovesandtime.csv'
data = pd.read_csv(file_path)

In [7]:
def get_player_history(player_name):
    player_moves_white = data[data['White'] == player_name][['WhiteMoves', 'BlackMoves']].copy()
    player_moves_black = data[data['Black'] == player_name][['BlackMoves', 'WhiteMoves']].rename(columns={'BlackMoves': 'WhiteMoves', 'WhiteMoves': 'BlackMoves'})
    
    # Use pd.concat to combine the DataFrames
    player_moves = pd.concat([player_moves_white, player_moves_black], ignore_index=True)
    
    # Flatten the moves
    player_moves['AllMoves'] = player_moves.apply(lambda row: f"{row['WhiteMoves']} {row['BlackMoves']}", axis=1)
    return player_moves['AllMoves'].tolist()

In [8]:
# Function to prepare data for model training
def prepare_data(player_name):
    player_moves_history = get_player_history(player_name)
    
    # Prepare features and target based on player moves
    move_sequences = []
    targets = []
    
    for moves in player_moves_history:
        moves_list = eval(moves)  # Convert string representation of list to actual list
        for i in range(len(moves_list) - 1):
            move_sequences.append(' '.join(moves_list[:i + 1]))
            targets.append(moves_list[i + 1])
    
    return move_sequences, targets

In [9]:
# Input: Player Name
input_player = "Magnus Carlsen"  # Replace with the actual player name you want to test

In [10]:

# Prepare data for the input player
X, y = prepare_data(input_player)

In [11]:
# Label Encoding the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [12]:
# Transforming the text data into numerical format using CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x.split())
X_vect = vectorizer.fit_transform(X)

c:\Users\shaik\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [13]:
# Check if X is empty
if not X:
    print("No valid move sequences found for the specified player.")
else:
    # Label Encoding the target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Transforming the text data into numerical format using CountVectorizer
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split())
    X_vect = vectorizer.fit_transform(X)

    # Initialize and train the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_vect, y_encoded)

    # Function to predict the next move for the input player
    def predict_next_move(player_name, opponent_move):
        move_history = get_player_history(player_name)
        last_moves = ' '.join(move_history[-1]) + f" {opponent_move}"  # Append the opponent's last move
        last_move_vect = vectorizer.transform([last_moves])
        
        predicted_move_index = rf_model.predict(last_move_vect)[0]
        predicted_move = label_encoder.inverse_transform([predicted_move_index])
        
        return predicted_move[0] if predicted_move else None

    # Example: Predict the next move for the input player given an opponent's last move
    opponent_last_move = "e4"  # Replace with the actual opponent's last move
    predicted_move = predict_next_move(input_player, opponent_last_move)
    print(f"Predicted next move for {input_player} after opponent's move {opponent_last_move}: {predicted_move}")

No valid move sequences found for the specified player.
